In [1]:
import numpy as np

#ポケモン一体の状態ベクトル化
def encode_pokemon(pokemon):
    #pokemonは辞書型で受け取る
    species_dict = {'ピカチュウ': 0, 'リザードン': 1, 'カビゴン': 2} #種族ID割り振り

    species_id = species_dict.get(pokemon['species'], -1)
    hp_ratio = pokemon['hp_ratio']
    status_map = {'なし': 0, 'まひ': 1, 'やけど': 2}
    status_id = status_map.get(pokemon['status'], 0)
    pp = np.array(pokemon['pp']) / 15  # PP最大15として正規化

    # ベクトル結合
    vector = np.array([species_id, hp_ratio, status_id])
    vector = np.concatenate([vector, pp])
    return vector

# テスト
poke_example = {'species': 'ピカチュウ', 'hp_ratio': 0.8, 'status': 'なし', 'pp': [5, 10, 10, 15]}
print(encode_pokemon(poke_example))

[0.         0.8        0.         0.33333333 0.66666667 0.66666667
 1.        ]
